In [14]:
import random
import math
from collections import defaultdict

def calculate_cost(grid, t_list):
    """计算当前布局的总传输成本"""
    positions = defaultdict(list)
    for idx in range(16):
        x = idx // 4
        y = idx % 4
        num = grid[idx]
        positions[num].append((x, y))
    
    cost = 0
    for i in range(11):  # 遍历所有可能的数值对（0-1到10-11）
        current_num = i
        next_num = i + 1
        if current_num not in positions or next_num not in positions:
            continue
        
        # 获取对应传输量（处理可能超出传输列表长度的情况）
        transmission = t_list[i] if i < len(t_list) else 0
        
        # 计算所有当前站点到下一站点的距离总和
        total_distance = 0
        for (x1, y1) in positions[current_num]:
            for (x2, y2) in positions[next_num]:
                total_distance += abs(x1 - x2) + abs(y1 - y2)
        
        cost += transmission * total_distance
    return cost

def generate_initial_solution(numbers):
    """生成初始随机解"""
    shuffled = numbers.copy()
    random.shuffle(shuffled)
    return shuffled

def get_neighbor(solution):
    """通过交换两个随机位置生成邻居解"""
    neighbor = solution.copy()
    i, j = random.sample(range(16), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def optimize_layout(numbers, t_list, initial_temp=10000, cooling_rate=0.995, iterations=5000):
    """模拟退火算法优化布局"""
    current_solution = generate_initial_solution(numbers)
    current_cost = calculate_cost(current_solution, t_list)
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    temp = initial_temp
    
    for _ in range(iterations):
        neighbor = get_neighbor(current_solution)
        neighbor_cost = calculate_cost(neighbor, t_list)
        
        # 决定是否接受新解
        if neighbor_cost < current_cost or random.random() < math.exp((current_cost - neighbor_cost) / temp):
            current_solution = neighbor
            current_cost = neighbor_cost
            
            if neighbor_cost < best_cost:
                best_solution = neighbor.copy()
                best_cost = neighbor_cost
        
        # 降温
        temp *= cooling_rate
    
    # 将最佳解转换为4x4网格
    grid = [best_solution[i*4 : (i+1)*4] for i in range(4)]
    return grid, best_cost

# 示例使用
if __name__ == "__main__":
    # 输入数据
    numbers = [0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11]
    transmission_volumes = [5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 7]  # 示例传输量列表（需要根据实际情况调整）
    
    # 优化布局
    best_grid, min_cost = optimize_layout(numbers, transmission_volumes)
    
    # 输出结果
    print(f"最小传输成本: {min_cost}")
    print("优化后的布局:")
    for row in best_grid:
        print(row)

最小传输成本: 133
优化后的布局:
[6, 5, 4, 3]
[7, 10, 11, 2]
[8, 9, 10, 1]
[7, 8, 9, 0]


In [11]:
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11]
generate_initial_solution(numbers)

[2, 10, 5, 4, 11, 9, 10, 8, 7, 0, 6, 8, 9, 7, 1, 3]

In [3]:
import random
import math
import numpy as np
from collections import defaultdict

# 通用成本计算函数
def calculate_cost(grid, t_list):
    positions = defaultdict(list)
    for idx in range(16):
        x = idx // 4
        y = idx % 4
        num = grid[idx]
        positions[num].append((x, y))
    
    cost = 0
    for i in range(11):
        current_num = i
        next_num = i + 1
        transmission = t_list[i] if i < len(t_list) else 0
        
        if current_num not in positions or next_num not in positions:
            continue
            
        total_distance = 0
        for (x1, y1) in positions[current_num]:
            for (x2, y2) in positions[next_num]:
                total_distance += abs(x1 - x2) + abs(y1 - y2)
        
        cost += transmission * total_distance
    return cost

# ================== 模拟退火算法 ==================
def sa_optimize(numbers, t_list, initial_temp=10000, cooling_rate=0.995, iterations=5000):
    current = numbers.copy()
    random.shuffle(current)
    best_solution = current.copy()
    best_cost = calculate_cost(current, t_list)
    
    temp = initial_temp
    
    for _ in range(iterations):
        neighbor = current.copy()
        i, j = random.sample(range(16), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        current_cost = calculate_cost(current, t_list)
        neighbor_cost = calculate_cost(neighbor, t_list)
        
        if neighbor_cost < current_cost or random.random() < math.exp((current_cost - neighbor_cost)/temp):
            current = neighbor
            current_cost = neighbor_cost
            
            if current_cost < best_cost:
                best_solution = current.copy()
                best_cost = current_cost
        
        temp *= cooling_rate
    
    return best_solution, best_cost

# ================== 遗传算法 ==================
class GeneticOptimizer:
    def __init__(self, numbers, t_list, pop_size=100, elite_ratio=0.2, mutation_rate=0.1):
        self.numbers = numbers
        self.t_list = t_list
        self.pop_size = pop_size
        self.elite_size = int(pop_size * elite_ratio)
        self.mutation_rate = mutation_rate
        
    def _initialize_population(self):
        population = []
        for _ in range(self.pop_size):
            individual = self.numbers.copy()
            random.shuffle(individual)
            population.append(individual)
        return population
    
    def _rank_individuals(self, population):
        costs = [(ind, calculate_cost(ind, self.t_list)) for ind in population]
        return sorted(costs, key=lambda x: x[1])
    
    def _select_parents(self, ranked_pop):
        weights = [1/(i+1) for i in range(len(ranked_pop))]
        return random.choices(
            [ind[0] for ind in ranked_pop],
            weights=weights,
            k=2
        )
    
    def _crossover(self, parent1, parent2):
        # 两点交叉
        point1 = random.randint(0, 15)
        point2 = random.randint(point1, 15)
        
        child = [None]*16
        remaining = []
        
        # 复制父代基因
        child[point1:point2] = parent1[point1:point2]
        
        # 处理重复元素
        count = defaultdict(int)
        for num in child[point1:point2]:
            count[num] += 1
        
        # 收集剩余元素
        for num in parent2:
            if count[num] > 0:
                count[num] -= 1
            else:
                remaining.append(num)
                
        # 填充空缺
        ptr = 0
        for i in range(16):
            if i < point1 or i >= point2:
                if ptr < len(remaining):
                    child[i] = remaining[ptr]
                    ptr += 1
        return child
    
    def _mutate(self, individual):
        if random.random() < self.mutation_rate:
            i, j = random.sample(range(16), 2)
            individual[i], individual[j] = individual[j], individual[i]
        return individual
    
    def optimize(self, generations=200):
        population = self._initialize_population()
        ranked = self._rank_individuals(population)
        best_solution = ranked[0][0].copy()
        best_cost = ranked[0][1]
        
        for _ in range(generations):
            new_population = []
            
            # 保留精英
            elites = [ind[0] for ind in ranked[:self.elite_size]]
            new_population.extend(elites)
            
            # 生成后代
            while len(new_population) < self.pop_size:
                parent1, parent2 = self._select_parents(ranked)
                child = self._crossover(parent1, parent2)
                child = self._mutate(child)
                new_population.append(child)
            
            population = new_population
            ranked = self._rank_individuals(population)
            
            if ranked[0][1] < best_cost:
                best_solution = ranked[0][0].copy()
                best_cost = ranked[0][1]
        
        return best_solution, best_cost

# ================== 综合比较 ==================
def compare_algorithms(numbers, t_list):
    # 运行模拟退火
    print("Running Simulated Annealing...")
    sa_solution, sa_cost = sa_optimize(numbers, t_list)
    
    # 运行遗传算法
    print("\nRunning Genetic Algorithm...")
    ga = GeneticOptimizer(numbers, t_list)
    ga_solution, ga_cost = ga.optimize()
    
    # 比较结果
    print("\n=== Comparison Results ===")
    print(f"SA Best Cost: {sa_cost}")
    print(f"GA Best Cost: {ga_cost}")
    
    # 选择更优解
    best_solution = sa_solution if sa_cost < ga_cost else ga_solution
    best_cost = min(sa_cost, ga_cost)
    
    # 格式化输出
    grid = [best_solution[i*4 : (i+1)*4] for i in range(4)]
    return grid, best_cost

# 示例使用
if __name__ == "__main__":
    numbers = [0,1,2,3,4,5,6,7,7,8,8,9,9,10,10,11]
    transmission_volumes = [5,4,3,2,1,2,3,4,5,6,7]  # 示例传输量
    
    best_grid, min_cost = compare_algorithms(numbers, transmission_volumes)
    
    print(f"\nMinimum Cost: {min_cost}")
    print("Optimized Layout:")
    for row in best_grid:
        print(row)

Running Simulated Annealing...

Running Genetic Algorithm...

=== Comparison Results ===
SA Best Cost: 137
GA Best Cost: 153

Minimum Cost: 137
Optimized Layout:
[6, 7, 8, 7]
[5, 10, 9, 8]
[0, 11, 10, 9]
[1, 2, 3, 4]


In [7]:
import random
import math
import numpy as np
from collections import defaultdict

# 通用成本计算函数
def calculate_cost(grid, t_list):
    positions = defaultdict(list)
    for idx in range(16):
        x = idx // 4
        y = idx % 4
        num = grid[idx]
        positions[num].append((x, y))
    
    cost = 0
    for i in range(11):
        current_num = i
        next_num = i + 1
        transmission = t_list[i] if i < len(t_list) else 0
        
        if current_num not in positions or next_num not in positions:
            continue
            
        total_distance = 0
        for (x1, y1) in positions[current_num]:
            for (x2, y2) in positions[next_num]:
                total_distance += abs(x1 - x2) + abs(y1 - y2)
        
        cost += transmission * total_distance
    return cost

# ================== 模拟退火算法 ==================
def sa_optimize(numbers, t_list, initial_temp=10000, cooling_rate=0.995, iterations=5000):
    current = numbers.copy()
    random.shuffle(current)
    best_solution = current.copy()
    best_cost = calculate_cost(current, t_list)
    
    temp = initial_temp
    
    for _ in range(iterations):
        neighbor = current.copy()
        i, j = random.sample(range(16), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        current_cost = calculate_cost(current, t_list)
        neighbor_cost = calculate_cost(neighbor, t_list)
        
        if neighbor_cost < current_cost or random.random() < math.exp((current_cost - neighbor_cost)/temp):
            current = neighbor
            current_cost = neighbor_cost
            
            if current_cost < best_cost:
                best_solution = current.copy()
                best_cost = current_cost
        
        temp *= cooling_rate
    
    return best_solution, best_cost

# ================== 遗传算法 ==================
class GeneticOptimizer:
    def __init__(self, numbers, t_list, pop_size=100, elite_ratio=0.2, mutation_rate=0.1):
        self.numbers = numbers
        self.t_list = t_list
        self.pop_size = pop_size
        self.elite_size = int(pop_size * elite_ratio)
        self.mutation_rate = mutation_rate
        
    def _initialize_population(self):
        population = []
        for _ in range(self.pop_size):
            individual = self.numbers.copy()
            random.shuffle(individual)
            population.append(individual)
        return population
    
    def _rank_individuals(self, population):
        costs = [(ind, calculate_cost(ind, self.t_list)) for ind in population]
        return sorted(costs, key=lambda x: x[1])
    
    def _select_parents(self, ranked_pop):
        weights = [1/(i+1) for i in range(len(ranked_pop))]
        return random.choices(
            [ind[0] for ind in ranked_pop],
            weights=weights,
            k=2
        )
    
    def _crossover(self, parent1, parent2):
        # 两点交叉
        point1 = random.randint(0, 15)
        point2 = random.randint(point1, 15)
        
        child = [None]*16
        remaining = []
        
        # 复制父代基因
        child[point1:point2] = parent1[point1:point2]
        
        # 处理重复元素
        count = defaultdict(int)
        for num in child[point1:point2]:
            count[num] += 1
        
        # 收集剩余元素
        for num in parent2:
            if count[num] > 0:
                count[num] -= 1
            else:
                remaining.append(num)
                
        # 填充空缺
        ptr = 0
        for i in range(16):
            if i < point1 or i >= point2:
                if ptr < len(remaining):
                    child[i] = remaining[ptr]
                    ptr += 1
        return child
    
    def _mutate(self, individual):
        if random.random() < self.mutation_rate:
            i, j = random.sample(range(16), 2)
            individual[i], individual[j] = individual[j], individual[i]
        return individual
    
    def optimize(self, generations=200):
        population = self._initialize_population()
        ranked = self._rank_individuals(population)
        best_solution = ranked[0][0].copy()
        best_cost = ranked[0][1]
        
        for _ in range(generations):
            new_population = []
            
            # 保留精英
            elites = [ind[0] for ind in ranked[:self.elite_size]]
            new_population.extend(elites)
            
            # 生成后代
            while len(new_population) < self.pop_size:
                parent1, parent2 = self._select_parents(ranked)
                child = self._crossover(parent1, parent2)
                child = self._mutate(child)
                new_population.append(child)
            
            population = new_population
            ranked = self._rank_individuals(population)
            
            if ranked[0][1] < best_cost:
                best_solution = ranked[0][0].copy()
                best_cost = ranked[0][1]
        
        return best_solution, best_cost

# ================== 蚁群算法 ==================
class AntColonyOptimizer:
    def __init__(self, numbers, t_list, num_ants=30, evaporation=0.1, alpha=1, beta=2, iterations=100):
        self.numbers = np.array(numbers)
        self.t_list = t_list
        self.num_ants = num_ants
        self.evaporation = evaporation
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
        # 信息素矩阵（位置 x 数值）
        self.pheromone = np.ones((16, 12)) * 0.1  # 数值范围0-11
        
    def _create_solution(self):
        solution = []
        available = list(range(16))
        for pos in range(16):
            probs = []
            for idx in available:
                num = self.numbers[idx]
                phe = self.pheromone[pos][num]
                count = self.numbers[available].tolist().count(num)
                probs.append((phe**self.alpha) * ((1/count)**self.beta))
            probs /= np.sum(probs)
            choice = np.random.choice(available, p=probs)
            solution.append(self.numbers[choice])
            available.remove(choice)
        return solution
    
    def optimize(self):
        best_sol = None
        best_cost = float('inf')
        
        for _ in range(self.iterations):
            solutions = []
            for _ in range(self.num_ants):
                sol = self._create_solution()
                cost = calculate_cost(sol, self.t_list)
                solutions.append((sol, cost))
                if cost < best_cost:
                    best_sol, best_cost = sol, cost
            
            # 更新信息素
            self.pheromone *= (1 - self.evaporation)
            for sol, cost in solutions:
                if cost == 0: continue
                delta = 1 / cost
                for pos in range(16):
                    num = sol[pos]
                    self.pheromone[pos][num] += delta
        
        return best_sol, best_cost

# ================== 粒子群算法 ==================
class ParticleSwarmOptimizer:
    def __init__(self, numbers, t_list, num_particles=50, w=0.7, c1=1.5, c2=1.5, max_iter=200):
        self.numbers = numbers
        self.t_list = t_list
        self.num_particles = num_particles
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.max_iter = max_iter
        
        # 初始化粒子位置（使用随机键编码）
        self.particles = [{
            'position': np.random.permutation(numbers),
            'velocity': np.zeros(16),
            'best_pos': None,
            'best_cost': float('inf')
        } for _ in range(num_particles)]
        
        self.global_best = None
        self.global_cost = float('inf')
    
    def optimize(self):
        for _ in range(self.max_iter):
            for p in self.particles:
                cost = calculate_cost(p['position'], self.t_list)
                
                if cost < p['best_cost']:
                    p['best_pos'] = p['position'].copy()
                    p['best_cost'] = cost
                
                if cost < self.global_cost:
                    self.global_best = p['position'].copy()
                    self.global_cost = cost
            
            for p in self.particles:
                # 离散速度更新
                r1 = random.random()
                r2 = random.random()
                new_vel = (self.w * p['velocity'] + 
                          self.c1 * r1 * (p['best_pos'] != p['position']) +
                          self.c2 * r2 * (self.global_best != p['position']))
                
                # 位置更新（交换操作）
                swap_idx = np.where(new_vel > 0.5)[0]
                for i in swap_idx:
                    j = random.randint(0, 15)
                    p['position'][i], p['position'][j] = p['position'][j], p['position'][i]
        
        return self.global_best.tolist(), self.global_cost

# ================== 综合比较 ==================
def compare_algorithms(numbers, t_list):
    print("Running Simulated Annealing...")
    sa_sol, sa_cost = sa_optimize(numbers, t_list)
    
    print("\nRunning Genetic Algorithm...")
    ga = GeneticOptimizer(numbers, t_list)
    ga_sol, ga_cost = ga.optimize()
    
    print("\nRunning Ant Colony Optimization...")
    aco = AntColonyOptimizer(numbers, t_list)
    aco_sol, aco_cost = aco.optimize()
    
    print("\nRunning Particle Swarm Optimization...")
    pso = ParticleSwarmOptimizer(numbers, t_list)
    pso_sol, pso_cost = pso.optimize()
    
    # 收集结果
    results = [
        ('SA', sa_sol, sa_cost),
        ('GA', ga_sol, ga_cost),
        ('ACO', aco_sol, aco_cost),
        ('PSO', pso_sol, pso_cost)
    ]
    
    # 选择最优解
    best = min(results, key=lambda x: x[2])
    
    print("\n=== Algorithm Performance ===")
    for name, _, cost in results:
        print(f"{name:5} Cost: {cost}")
    
    print(f"\nBest solution from {best[0]} with cost {best[2]}")
    return [best[1][i*4:(i+1)*4] for i in range(4)], best[2]

# 示例使用
if __name__ == "__main__":
    numbers = [0,1,2,3,4,5,6,7,7,8,8,9,9,10,10,11]
    t_list = [5,4,3,2,1,2,3,4,5,6,7]  # 0-1到10-11的传输量
    
    best_grid, min_cost = compare_algorithms(numbers, t_list)
    
    print("\nOptimized Layout:")
    for row in best_grid:
        print(row)

Running Simulated Annealing...

Running Genetic Algorithm...

Running Ant Colony Optimization...

Running Particle Swarm Optimization...

=== Algorithm Performance ===
SA    Cost: 135
GA    Cost: 139
ACO   Cost: 155
PSO   Cost: 163

Best solution from SA with cost 135

Optimized Layout:
[11, 10, 4, 5]
[10, 9, 9, 8]
[2, 3, 8, 7]
[1, 0, 7, 6]
